In [ ]:
pip install nltk

In [ ]:
pip install textblob

In [ ]:
pip install vaderSentiment

In [20]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon
# nltk.download('vader_lexicon')
# nltk.download('vader_lexicon', download_dir='/usr/local/nltk_data')

sia = SentimentIntensityAnalyzer()

In [11]:
drive_path = '/Users/danperez/Library/CloudStorage/OneDrive-ClaremontGraduateUniversity/332 NLP/Assignments/Final/longformer.csv'
longformer_df = pd.read_csv(drive_path)
longformer_df.head()

,recommendationid,Appname,review_text,0,1,2,3,4,5,6,...,758,759,760,761,762,763,764,765,766,767
0,212664845,ARC Raiders,Addictive. Stressful. Time waster.,0.160998,-0.314331,0.128970,-0.002612,0.363178,-0.224010,-0.496514,...,-0.242919,0.057376,0.166702,0.129182,-0.262234,0.042280,-0.297840,-0.106902,0.149733,0.019978
1,212664705,ARC Raiders,If the Steam comments section is like every ot...,0.171792,-0.321767,0.087026,-0.013351,0.341068,-0.235470,-0.495652,...,-0.243866,0.088027,0.174401,0.125420,-0.211613,0.038621,-0.270587,-0.055174,0.198233,0.028517
2,212664692,ARC Raiders,I like the gathering and sneaking around the A...,0.172855,-0.294004,0.098145,-0.008861,0.330265,-0.219347,-0.502270,...,-0.233194,0.068008,0.172816,0.134451,-0.217051,0.045791,-0.247482,-0.063542,0.169679,0.000969
3,212664560,ARC Raiders,"Very well made game, every time I hop on I exp...",0.192783,-0.339317,0.094364,0.009529,0.344692,-0.200341,-0.490265,...,-0.261185,0.022664,0.213612,0.160738,-0.223490,0.026880,-0.263800,-0.075613,0.200625,0.042513
4,212664471,ARC Raiders,I thought this would be too sweaty for me. Hon...,0.166418,-0.311148,0.098709,-0.004635,0.343317,-0.196987,-0.491329,...,-0.209657,0.029193,0.166780,0.151393,-0.223100,0.029565,-0.267660,-0.060990,0.182768,0.028096


In [22]:
def compute_sentiment(row):
    text = str(row["review_text"]).strip()

    if not text:
        return pd.Series({
            "Polarity": 0,
            "Subjectivity": 0,
            "NLTK_Compound": 0,
            "Sentiment_Label": "Neutral"
        })

    blob = TextBlob(text)
    
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    compound = sia.polarity_scores(text)["compound"]

    # Sentiment category
    if compound >= 0.05:
        label = "Positive"
    elif compound <= -0.05:
        label = "Negative"
    else:
        label = "Neutral"

    return pd.Series({
        "Polarity": polarity,
        "Subjectivity": subjectivity,
        "NLTK_Compound": compound,
        "Sentiment_Label": label
    })

sent_df = longformer_df.apply(compute_sentiment, axis=1)
sent_df.head()

,Polarity,Subjectivity,NLTK_Compound,Sentiment_Label
0,0.000000,0.900000,-0.5106,Negative
1,0.022209,0.486415,-0.9429,Negative
2,-0.275000,0.250000,-0.2498,Negative
3,-0.021212,0.384848,0.3384,Positive
4,0.252083,0.510417,0.5568,Positive


In [26]:
longformer_sentiment_df = pd.concat([longformer_df['recommendationid'], sent_df], axis=1)

# Save the final dataframe to a CSV file
longformer_sentiment_df.to_csv("longformer_with_sentiment.csv", index=False)

longformer_sentiment_df.head()

,recommendationid,Polarity,Subjectivity,NLTK_Compound,Sentiment_Label
0,212664845,0.000000,0.900000,-0.5106,Negative
1,212664705,0.022209,0.486415,-0.9429,Negative
2,212664692,-0.275000,0.250000,-0.2498,Negative
3,212664560,-0.021212,0.384848,0.3384,Positive
4,212664471,0.252083,0.510417,0.5568,Positive


## Average sentiment per game

In [25]:

game_sentiment_avg = (
    longformer_sentiment_df
    .groupby("Appname")
    .agg({
        "Polarity": "mean",
        "Subjectivity": "mean",
        "NLTK_Compound": "mean"
    })
    .reset_index()
)

# Add sentiment label for each game based on avg compound
def game_sentiment_label(compound):
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    return "Neutral"

game_sentiment_avg["Game_Sentiment_Label"] = game_sentiment_avg["NLTK_Compound"].apply(game_sentiment_label)

# Save the final dataframe to a CSV file
game_sentiment_avg.to_csv("game_sentiment.csv", index=False)

game_sentiment_avg.head()


,Appname,Polarity,Subjectivity,NLTK_Compound,Game_Sentiment_Label
0,7 Days to Die,0.117871,0.458729,0.509609,Positive
1,A Total War Saga: TROY,0.064720,0.509241,0.214725,Positive
2,ARC Raiders,0.077626,0.465590,0.252593,Positive
3,ARK: Survival Ascended,0.057032,0.465810,0.349379,Positive
4,Abiotic Factor,0.151070,0.481788,0.560700,Positive
